In [ ]:
import requests
from bs4 import BeautifulSoup
import os

url = 'http://data.gdeltproject.org/events/index.html'

download_dir = '../data/downloads'

if not os.path.exists(download_dir):
    os.makedirs(download_dir)

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

zip_links = soup.find_all('a', href=True)
zip_links = [link['href'] for link in zip_links if link['href'].endswith('.zip')]

from tqdm.notebook import tqdm
import re
import datetime

threshold_date = datetime.datetime.strptime('200803', '%Y%m')

for link in tqdm(zip_links):
    if link[0] not in ['2'] and link[2] not in ['0'] and link[3] not in ['0', '1', '2', '3', '4', '5']:
        continue

    match = re.search(r'(\d{4})\.zip$', link)
    if match:
        file_date_str = match.group(1)
        file_date = datetime.datetime.strptime(file_date_str, '%Y')
        
        if file_date < threshold_date:
            if not link.startswith('http'):
                link = requests.compat.urljoin(url, link)
            
            try:
                file_name = os.path.join(download_dir, os.path.basename(link))
                
                with requests.get(link, stream=True) as r:
                    r.raise_for_status()
                    with open(file_name, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)
                print(f'Saved to {file_name}')
            except Exception as e:
                print(f'Error downloading {link}: {e}')
        else:
            print(f'Skipping {link}: File date is not before {threshold_date.strftime("%Y-%m-%d")}.')
    else:
        print(f'Skipping {link}: Unable to extract date from filename.')

In [1]:
import zipfile
import os

download_dir = '../data/downloads'

unpack_dir = '../data/unpacked'

if not os.path.exists(unpack_dir):
    os.makedirs(unpack_dir)

zip_files = [f for f in os.listdir(download_dir) if f.endswith('.zip')]

for zip_file in zip_files:
    zip_path = os.path.join(download_dir, zip_file)
    if int(zip_file.split('.')[0]) > 20160203:
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(unpack_dir)
            print(f'Unpacked {zip_file} to {unpack_dir}')
        except zipfile.BadZipFile:
            print(f'Error: {zip_file} is not a valid zip file')
        except Exception as e:
            print(f'Error unpacking {zip_file}: {e}')
    else:
        print(f"Skipped {zip_file}")

Skipped 1979.zip
Skipped 1980.zip
Skipped 1981.zip
Skipped 1982.zip
Skipped 1983.zip
Skipped 1984.zip
Skipped 1985.zip
Skipped 1986.zip
Skipped 1987.zip
Skipped 1988.zip
Skipped 1989.zip
Skipped 1990.zip
Skipped 1991.zip
Skipped 1992.zip
Skipped 1993.zip
Skipped 1994.zip
Skipped 1995.zip
Skipped 1996.zip
Skipped 1997.zip
Skipped 1998.zip
Skipped 1999.zip
Skipped 2000.zip
Skipped 2001.zip
Skipped 2002.zip
Skipped 2003.zip
Skipped 2004.zip
Skipped 2005.zip
Skipped 200601.zip
Skipped 200602.zip
Skipped 200603.zip
Skipped 200604.zip
Skipped 200605.zip
Skipped 200606.zip
Skipped 200607.zip
Skipped 200608.zip
Skipped 200609.zip
Skipped 200610.zip
Skipped 200611.zip
Skipped 200612.zip
Skipped 200701.zip
Skipped 200702.zip
Skipped 200703.zip
Skipped 200704.zip
Skipped 200705.zip
Skipped 200706.zip
Skipped 200707.zip
Skipped 200708.zip
Skipped 200709.zip
Skipped 200710.zip
Skipped 200711.zip
Skipped 200712.zip
Skipped 200801.zip
Skipped 200802.zip
Skipped 200803.zip
Skipped 200804.zip
Skipped 2

In [21]:
import os
import pandas as pd

folder_path = '../data/unpacked/'

column_names = [
    'GLOBALEVENTID', 'SQLDATE', 'MonthYear', 'Year', 'FractionDate', 'Actor1Code',
    'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode', 'Actor1EthnicCode',
    'Actor1Religion1Code', 'Actor1Religion2Code', 'Actor1Type1Code', 'Actor1Type2Code',
    'Actor1Type3Code', 'Actor2Code', 'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode',
    'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code', 'Actor2Type1Code',
    'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent', 'EventCode', 'EventBaseCode',
    'EventRootCode', 'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources',
    'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_FullName', 'Actor1Geo_CountryCode',
    'Actor1Geo_ADM1Code', 'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor1Geo_FeatureID',
    'Actor2Geo_Type', 'Actor2Geo_FullName', 'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code',
    'Actor2Geo_Lat', 'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type',
    'ActionGeo_FullName', 'ActionGeo_CountryCode', 'ActionGeo_ADM1Code', 'ActionGeo_Lat',
    'ActionGeo_Long', 'ActionGeo_FeatureID', 'DATEADDED', 'URL'
]

selected_columns = [
    'SQLDATE', 'MonthYear', 'Actor1Code', 'Actor1CountryCode', 'Actor1Type1Code',
    'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code', 'Actor2CountryCode',
    'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent',
    'EventCode', 'EventRootCode', 'QuadClass', 'GoldsteinScale', 'NumMentions',
    'NumSources', 'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_CountryCode',
    'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor2Geo_Type', 'Actor2Geo_CountryCode',
    'Actor2Geo_Lat', 'Actor2Geo_Long', 'ActionGeo_Type', 'ActionGeo_Lat', 'ActionGeo_Long'
]
csv_files = [file for file in os.listdir(folder_path) if file.lower().endswith('.csv')]

for file_name in tqdm(csv_files, desc="Loading CSV files into MySQL", unit="file"):
    if file_name.lower().endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        output_file_path = f'../data/reduced/{file_name}'

        df = pd.read_csv(file_path, delimiter='\t', header=None, names=column_names)
        
        df_selected = df[selected_columns]

        df_selected.to_csv(output_file_path, index=False)
        
        print(f'{file_name} został przetworzony i zapisany jako {os.path.basename(output_file_path)}')


Loading CSV files into MySQL:   0%|          | 0/4182 [00:00<?, ?file/s]C:\Users\tomek\AppData\Local\Temp/ipykernel_11380/2472952172.py:42: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter='\t', header=None, names=column_names)
Loading CSV files into MySQL:   0%|          | 1/4182 [00:09<10:57:54,  9.44s/file]

1979.csv został przetworzony i zapisany jako 1979.csv


Loading CSV files into MySQL:   0%|          | 2/4182 [00:21<12:55:04, 11.13s/file]

1980.csv został przetworzony i zapisany jako 1980.csv


Loading CSV files into MySQL:   0%|          | 3/4182 [00:35<14:30:27, 12.50s/file]

...

20240531.export.CSV został przetworzony i zapisany jako 20240531.export.CSV


Loading CSV files into MySQL: 100%|█████████▉| 4181/4182 [5:55:35<00:02,  2.78s/file]

20240601.export.CSV został przetworzony i zapisany jako 20240601.export.CSV


Loading CSV files into MySQL: 100%|██████████| 4182/4182 [5:55:37<00:00,  5.10s/file]

20240602.export.CSV został przetworzony i zapisany jako 20240602.export.CSV
